In [3]:
import pandas as pd
from sklearn import tree,cross_validation
import sklearn.ensemble as ek
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
print(pd.__version__)

0.25.1


In [4]:
#读取特征集
featureSet = pd.read_csv("../data/features.csv")

In [5]:
len(featureSet)

14157

In [6]:
featureSet.groupby(featureSet['label']).size()
#0为非恶意url
#1为恶意url

label
0    12445
1     1712
dtype: int64

In [7]:
X = featureSet.drop(['url','label'],axis=1).values
y = featureSet['label'].values

In [8]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y ,test_size=0.3)

In [9]:

model = { "DecisionTree":tree.DecisionTreeClassifier(max_depth=10),
         "RandomForest":ek.RandomForestClassifier(n_estimators=50),
         "Adaboost":ek.AdaBoostClassifier(n_estimators=50),
         "GradientBoosting":ek.GradientBoostingClassifier(n_estimators=50),
         "GNB":GaussianNB(),
         "LogisticRegression":LogisticRegression()   
}


In [10]:
#测试不同算法的准确度
results = {}
for algo in model:
    clf = model[algo]
    clf.fit(X_train,y_train)
    score_test = clf.score(X_test,y_test)
    score_train = clf.score(X_train,y_train)
    print ("%s : %s %s" %(algo, score_test,score_train))
    results[algo] = score_test

DecisionTree : 0.9910546139359698 0.9906145927944293
RandomForest : 0.9910546139359698 0.9906145927944293
Adaboost : 0.9910546139359698 0.9905136744373801
GradientBoosting : 0.9912900188323918 0.9905136744373801
GNB : 0.9901129943502824 0.989100817438692
LogisticRegression : 0.9912900188323918 0.9905136744373801


In [11]:
print(results)

{'DecisionTree': 0.9910546139359698, 'RandomForest': 0.9910546139359698, 'Adaboost': 0.9910546139359698, 'GradientBoosting': 0.9912900188323918, 'GNB': 0.9901129943502824, 'LogisticRegression': 0.9912900188323918}


In [12]:
winner = max(results, key=results.get)
print(winner)

GradientBoosting


In [13]:
clf = model[winner]
res = clf.predict(X_test)
mt = confusion_matrix(y_test, res)#混淆矩阵
print("False positive rate : %f %%" % ((mt[0][1] / float(sum(mt[0])))*100))
print('False negative rate : %f %%' % ( (mt[1][0] / float(sum(mt[1]))*100)))

False positive rate : 0.000000 %
False negative rate : 7.101727 %


In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [15]:
#准确率 = 分类正确的样本数/总样本数
accuracy_score(y_test, res)

0.9912900188323918

In [16]:
#精确率 = 真阳性样本数量与所有被分类为阳性的样本的数量的比值
precision_score(y_test, res)

1.0

In [17]:
#召回率 = 为真阳性样本数量与样本集中全部阳性样本的数量的比值
recall_score(y_test, res)

0.9289827255278311

In [18]:
from sklearn.externals import joblib #jbolib模块

In [19]:
#保存训练好的模型
joblib.dump(clf, '../data/clf.pkl')


['../data/clf.pkl']

In [20]:
import get_features

In [21]:

result = pd.DataFrame(columns=('url','no of dots','presence of hyphen','len of url','presence of at',\
'presence of double slash','no of subdir','no of subdomain','len of domain','no of queries','is IP','presence of Suspicious_TLD',\
'presence of suspicious domain','label'))

results = get_features.getFeatures('www.baidu.com', '1')
result.loc[0] = results
result = result.drop(['url','label'],axis=1).values
print(clf.predict(result))

[0]
